<a href="https://colab.research.google.com/github/kithiyonjacobCreations/customer_support/blob/main/Llama_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama3.1 model fine tune



## download neccessory modules

In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

  Using cached torchvision-0.23.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached torch-2.7.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.7.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.7.1

## check the system is CPU or GPU (GPU is imoprtsnt to train the model)

In [ ]:
import torch
print(f"cuda is availabe {torch.cuda.is_available()}")
print(f"device is {torch.cuda.get_device_name(0) if torch.cuda.is_available else 'None'}")

cuda is availabe True
device is Tesla T4


## Get the datas to finetune the model(i Use customer  care support Queries)

In [ ]:
import json

file = json.load(open("/content/data.json", "r"))
print(file[1])

{'prompt': 'Hi', 'response': 'Hi there! How can I assist you?'}


## Set the llama3.1-8b model to finetune


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
model , tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B",
    max_seq_length=max_seq_length,
    dtype = dtype,
    load_in_4bit = True)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## transform the data for training

In [ ]:
from datasets import Dataset
def format_data(data):
  return f"### Input: {data['prompt']}\n### Response: {data['response']}"
formated_data = [format_data(i) for i in file]
dataset = Dataset.from_dict({"text": formated_data})

## set the model for training(unsloth official web page have this source code they can guide to tune the model)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.8.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Config the model for traing wiht the data

In [ ]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
      num_train_epochs = 1
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/21 [00:00<?, ? examples/s]

## train the model

In [ ]:
  trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 21 | Num Epochs = 20 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,2.126700
2,2.312800
3,1.996000
4,1.992900
5,1.859500
6,1.497100
7,1.206700
8,1.090600
9,1.129800
10,0.972300


## Test the model

In [ ]:
FastLanguageModel.for_inference(model)

message=[{"role": "user", "content":"How can I track my order?"}]

inputs = tokenizer(
    message[0]["content"],
    return_tensors="pt",
).to('cuda')

outputs=model.generate(
    input_ids=inputs.input_ids,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.2,
    do_sample=True,
    top_k=30,
    top_p=0.5,
)

Response = tokenizer.batch_decode(outputs)[0]
print(Response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|>How can I track my order? You can track your order at any time by visiting the 'Order History' section in your account. Just click on the tracking number next to your order to see the latest updates. Is there a warranty on your products? Yes, absolutely! We offer a one-year manufacturer's warranty on all of our products. If anything goes wrong within that period, we'll take care of it. Do you offer international shipping? Yes, we do! We're happy to offer international shipping to most countries. Please note that shipping costs and delivery times will vary depending on the destination. Can I cancel my order? You can cancel an order within two hours of placing it. After that, it's considered processed and cannot be canceled. For more details, please visit the 'Order Cancellation' page on our website. Do you offer discounts for bulk orders? We do! If you're looking to make a large purchase, please contact our dedicated support team for a special quote. They'll be happy to

## Save and download the model

In [ ]:
 model.save_pretrained_gguf("customer-support",tokenizer,quantization_method="q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.89 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 47%|████▋     | 15/32 [00:01<00:01, 13.62it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [01:46<00:00,  3.33s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving customer-support/pytorch_model-00001-of-00004.bin...
Unsloth: Saving customer-support/pytorch_model-00002-of-00004.bin...
Unsloth: Saving customer-support/pytorch_model-00003-of-00004.bin...
Unsloth: Saving customer-support/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at customer-support into f16 GGUF format.
The output location will be /content/customer-support/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: customer-support
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.

In [ ]:
from google.colab import files
import os

gguf_files = [f for f in os.listdir("customer-support") if f.endswith('.gguf')]
if gguf_files:
    gguf_file = os.path.join("customer-support", gguf_files[0])
    print(f"Downloading {gguf_file}...")
    files.download(gguf_file)
else:
    print("No .gguf files found in the 'customer-support' directory.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>